In [1]:
## library imports

## file IO
import pandas as pd

## db connection
import config

## linear algebra
import numpy as np

In [2]:
## db engine

engine = config.get_engine(database='processing')
validated_engine = config.get_engine(database='validated')

In [3]:
## read in scores
scores = pd.read_sql("SELECT * FROM Linescore;",con=engine)

In [4]:
## columns of interest

ends = ['End{}'.format(_) for _ in range(1,13)]

In [5]:
## Replace nulls with 0s and add up linescores

scores['FinalScoreCheck'] = scores[ends].replace(np.nan,0).sum(axis=1)

In [6]:
## Elimnate linescores where the noted final score does not equal the final score check

scores = scores.loc[scores.FinalScore == scores.FinalScoreCheck].reset_index(drop=True)

## drop unneeded column
scores = scores.drop("FinalScoreCheck",axis=1)

In [7]:
## Find games where sum of final score is greater than 0

grouped_fs = scores.groupby(['GameID'])[['FinalScore']].sum()
grouped_fs = grouped_fs.loc[grouped_fs.FinalScore > 0]

In [8]:
## Find games where valid hammer is present

grouped_hammer = scores.groupby(['GameID'])[['Hammer']].sum()
grouped_hammer = grouped_hammer.loc[grouped_hammer.Hammer == 1]

In [9]:
## filter scores down

## filter scores where final score is greater than 0
scores = scores.loc[scores.GameID.isin(grouped_fs.index)]

## filter scores where valid hammer exists 
scores = scores.loc[scores.GameID.isin(grouped_hammer.index)]

In [10]:
## sum up count of GameIDs

s = scores.GameID.value_counts()

## only keep GameIDs where count == 2 (2 linescores for 1 game )
s = s.loc[s == 2]

## filter games where 2 GameIDs exists
scores = scores.loc[scores.GameID.isin(s.index)]


In [11]:
## sql

index = False
if_exists = 'append'

scores.to_sql("Linescore",con=validated_engine,index=index,if_exists=if_exists)